code from https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py

In [22]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate, Conv2DTranspose, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.utils import plot_model
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Dataset

In [2]:
data = pd.read_csv('./fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
data.Usage.value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [4]:
num_classes = 7
img_width = 48
img_height = 48

In [5]:
X = data['pixels']
y = data['emotion']

X_train = []
for i in X:
    X_train.append([int(j) for j in i.split()])

X_train = np.array(X_train)/255.0

X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_train = X_train.astype('float32')

print(X_train.shape)

(35887, 48, 48, 1)


## Model Conditional GAN

In [158]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 7
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        # (_, y_train), (_, _) = mnist.load_data()

        # Configure input
        # X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y.to_numpy().reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 7, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    # def sample_images(self, epoch):
    #     r, c = 1, 7
    #     noise = np.random.normal(0, 1, (r * c, 100))
    #     sampled_labels = np.arange(0, 7).reshape(-1, 1)

    #     gen_imgs = self.generator.predict([noise, sampled_labels])
    #     print(gen_imgs.shape)

    #     # Rescale images 0 - 1
    #     gen_imgs = 0.5 * gen_imgs + 0.5

    #     fig, axs = plt.subplots(r, c)
    #     cnt = 0
    #     for i in range(r):
    #         print(i)
    #         for j in range(c):
    #             print(j)
    #             axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
    #             axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
    #             axs[i,j].axis('off')
    #             cnt += 1
    #     fig.savefig("images/%d.png" % epoch)
    #     plt.close()

    def sample_images(self, epoch):
          r, c = 1, 7
          noise = np.random.normal(0, 1, (r * c, 100))
          sampled_labels = np.arange(0, 7).reshape(-1, 1)

          gen_imgs = self.generator.predict([noise, sampled_labels])
          print(gen_imgs.shape)

          # Rescale images 0 - 1
          gen_imgs = 0.5 * gen_imgs + 0.5

          fig, axs = plt.subplots(r, c)
          cnt = 0
          for j in range(c):
              axs[j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
              axs[j].set_title("label: %d" % sampled_labels[cnt])
              axs[j].axis('off')
              cnt += 1
          fig.savefig("images/%d.png" % epoch)
          plt.close()




In [159]:
if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=10000, batch_size=128, sample_interval=200)

Streaming output truncated to the last 5000 lines.
5015 [D loss: 0.690500, acc.: 54.69%] [G loss: 0.703922]
5016 [D loss: 0.694686, acc.: 48.05%] [G loss: 0.705158]
5017 [D loss: 0.691771, acc.: 47.66%] [G loss: 0.704135]
5018 [D loss: 0.692664, acc.: 45.70%] [G loss: 0.705885]
5019 [D loss: 0.692847, acc.: 46.09%] [G loss: 0.708866]
5020 [D loss: 0.694201, acc.: 49.61%] [G loss: 0.706673]
5021 [D loss: 0.693590, acc.: 48.44%] [G loss: 0.703099]
5022 [D loss: 0.694926, acc.: 48.44%] [G loss: 0.701726]
5023 [D loss: 0.695554, acc.: 48.05%] [G loss: 0.701245]
5024 [D loss: 0.695010, acc.: 44.53%] [G loss: 0.699076]
5025 [D loss: 0.697263, acc.: 41.02%] [G loss: 0.706986]
5026 [D loss: 0.692163, acc.: 46.48%] [G loss: 0.703334]
5027 [D loss: 0.693365, acc.: 49.22%] [G loss: 0.709379]
5028 [D loss: 0.692909, acc.: 50.78%] [G loss: 0.696689]
5029 [D loss: 0.691308, acc.: 49.22%] [G loss: 0.699807]
5030 [D loss: 0.697001, acc.: 44.53%] [G loss: 0.698422]
5031 [D loss: 0.692161, acc.: 44.92%]